In [2]:
# !pip install tensorflow-datasets

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
# from keras.models import load_model
from datetime import datetime
tf.enable_eager_execution()

C:\Users\Affinity\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
W1023 20:34:49.376404 18464 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # 키워드 검색하기
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dropout (Dropout)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                81930     
Total params: 82,954
Trainable params: 82,890
Non-trainable params: 64
___________________________________________________

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(10000).batch(64) # 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [8]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_acc',
    save_best_only=True)

In [9]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [10]:
history = model.fit(train_data,
                    epochs=10,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

W1023 20:34:53.209528 18464 deprecation.py:323] From C:\Users\Affinity\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 0 ends at 20:36:56.914947: {'loss': 1.5966620832443237, 'acc': 0.45645, 'val_loss': 1.5100887732900632, 'val_acc': 0.4868}
Epoch 1 ends at 20:39:04.615714: {'loss': 1.2489439393997193, 'acc': 0.569625, 'val_loss': 1.1727690157616975, 'val_acc': 0.5924}
Epoch 2 ends at 20:41:12.605590: {'loss': 1.1461329936981202, 'acc': 0.602275, 'val_loss': 1.2461531055960686, 'val_acc': 0.575}
Epoch 3 ends at 20:43:20.136050: {'loss': 1.089291981124878, 'acc': 0.6236, 'val_loss': 1.1592662820390836, 'val_acc': 0.6034}
Epoch 4 ends at 20:45:27.747300: {'loss': 1.0528878829956054, 'acc': 0.6331, 'val_loss': 1.146697270262773, 'val_acc': 0.6141}
Epoch 5 ends at 20:47:35.198957: {'loss': 1.017085502052307, 'acc': 0.64815, 'val_loss': 1.183781301140026, 'val_acc': 0.5978}
Epoch 6 ends at 20:49:42.020394: {'loss': 0.9918185165405273, 'acc': 0.6543, 'val_loss': 1.203700229620478, 'val_acc': 0.5884}
Epoch 7 ends at 20:51:49.084088: {'loss': 0.9869338397979737, 'acc': 0.65755, 'val_loss': 1.135867841304

In [11]:
model.evaluate(test_data)

    157/Unknown - 0s 108ms/step - loss: 1.0677 - acc: 0.73 - 0s 78ms/step - loss: 1.0869 - acc: 0.6719 - 0s 63ms/step - loss: 1.1522 - acc: 0.666 - 0s 52ms/step - loss: 1.0916 - acc: 0.671 - 0s 46ms/step - loss: 1.0768 - acc: 0.675 - 0s 42ms/step - loss: 1.0911 - acc: 0.666 - 0s 40ms/step - loss: 1.0912 - acc: 0.662 - 0s 38ms/step - loss: 1.0981 - acc: 0.658 - 0s 36ms/step - loss: 1.1073 - acc: 0.652 - 0s 35ms/step - loss: 1.0978 - acc: 0.656 - 0s 33ms/step - loss: 1.1070 - acc: 0.656 - 0s 32ms/step - loss: 1.1477 - acc: 0.644 - 0s 31ms/step - loss: 1.1373 - acc: 0.644 - 0s 30ms/step - loss: 1.1256 - acc: 0.641 - 0s 29ms/step - loss: 1.1308 - acc: 0.637 - 0s 29ms/step - loss: 1.1426 - acc: 0.634 - 0s 28ms/step - loss: 1.1492 - acc: 0.628 - 0s 27ms/step - loss: 1.1512 - acc: 0.625 - 1s 27ms/step - loss: 1.1576 - acc: 0.621 - 1s 27ms/step - loss: 1.1702 - acc: 0.618 - 1s 27ms/step - loss: 1.1535 - acc: 0.622 - 1s 26ms/step - loss: 1.1516 - acc: 0.625 - 1s 26ms/step - loss: 1.1538 - acc: 

[1.1759216102065555, 0.606]

In [12]:
m = tf.keras.models.load_model('checkpoints/best.hdf5')

In [13]:
m.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dropout (Dropout)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                81930     
Total params: 82,954
Trainable params: 82,890
Non-trainable params: 64
___________________________________________________

In [14]:
m.evaluate(test_data)

    157/Unknown - 0s 229ms/step - loss: 1.4995 - acc: 0.50 - 0s 127ms/step - loss: 1.4675 - acc: 0.56 - 0s 93ms/step - loss: 1.5331 - acc: 0.5625 - 0s 76ms/step - loss: 1.4625 - acc: 0.574 - 0s 64ms/step - loss: 1.4089 - acc: 0.578 - 0s 58ms/step - loss: 1.3938 - acc: 0.575 - 0s 53ms/step - loss: 1.3986 - acc: 0.567 - 0s 49ms/step - loss: 1.4066 - acc: 0.562 - 0s 46ms/step - loss: 1.4087 - acc: 0.560 - 0s 44ms/step - loss: 1.4016 - acc: 0.560 - 0s 42ms/step - loss: 1.4158 - acc: 0.558 - 0s 40ms/step - loss: 1.4410 - acc: 0.544 - 0s 38ms/step - loss: 1.4386 - acc: 0.544 - 1s 36ms/step - loss: 1.4555 - acc: 0.539 - 1s 36ms/step - loss: 1.4599 - acc: 0.539 - 1s 35ms/step - loss: 1.4672 - acc: 0.537 - 1s 34ms/step - loss: 1.4756 - acc: 0.531 - 1s 33ms/step - loss: 1.4793 - acc: 0.526 - 1s 32ms/step - loss: 1.4836 - acc: 0.524 - 1s 32ms/step - loss: 1.4974 - acc: 0.517 - 1s 31ms/step - loss: 1.4655 - acc: 0.526 - 1s 31ms/step - loss: 1.4636 - acc: 0.526 - 1s 31ms/step - loss: 1.4568 - acc: 

[1.4784323979335225, 0.5119]

In [15]:
m.evaluate(valid_data)

    157/Unknown - 0s 433ms/step - loss: 1.9417 - acc: 0.48 - 0s 229ms/step - loss: 1.5641 - acc: 0.56 - 0s 159ms/step - loss: 1.5216 - acc: 0.55 - 0s 123ms/step - loss: 1.4327 - acc: 0.55 - 1s 103ms/step - loss: 1.4138 - acc: 0.55 - 1s 90ms/step - loss: 1.4655 - acc: 0.5339 - 1s 80ms/step - loss: 1.4854 - acc: 0.524 - 1s 72ms/step - loss: 1.4733 - acc: 0.525 - 1s 66ms/step - loss: 1.4693 - acc: 0.517 - 1s 61ms/step - loss: 1.4625 - acc: 0.517 - 1s 57ms/step - loss: 1.4947 - acc: 0.504 - 1s 55ms/step - loss: 1.4954 - acc: 0.506 - 1s 52ms/step - loss: 1.4911 - acc: 0.509 - 1s 50ms/step - loss: 1.5123 - acc: 0.503 - 1s 48ms/step - loss: 1.5227 - acc: 0.497 - 1s 47ms/step - loss: 1.5225 - acc: 0.499 - 1s 46ms/step - loss: 1.5072 - acc: 0.498 - 1s 45ms/step - loss: 1.5206 - acc: 0.496 - 1s 44ms/step - loss: 1.5086 - acc: 0.500 - 1s 43ms/step - loss: 1.5045 - acc: 0.498 - 1s 42ms/step - loss: 1.5103 - acc: 0.499 - 1s 41ms/step - loss: 1.5014 - acc: 0.503 - 1s 40ms/step - loss: 1.4913 - acc: 

[1.4847465670032867, 0.508]